In [ ]:
# @title Instalar dependencias

r = "Retrieval"
b = "based"
v = "Voice"
c = "Conversion"
w = "WebUI"

![ -d /content/rvc ] || git clone https://github.com/RVC-Project/{"-".join([r,b,v,c,w])}
!mv /content/{"-".join([r,b,v,c,w])} /content/rvc
%cd /content/rvc
!rm {"_".join([r,b,v,c,w])}*.ipynb

# Definir python 3.10 como predeterminado
!rm /usr/local/bin/python
!ln -s /bin/python3.10 /usr/local/bin/python
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!rm get-pip.py
!python -m pip install "pip<24.1"

!cat requirements.txt | grep -v {"".join(["g","r","a","d","i","o"])} > requirements2.txt
!pip install -q -r requirements2.txt
!pip uninstall -y torch torchaudio
!pip install torch==2.5.1 torchaudio --index-url https://download.pytorch.org/whl/cu118

# Descargar modelos pre-entrenados
aria2c = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
%cd assets
hugging_user = "".join(["l", "j", "1995"])
huggingface_repo = f"https://huggingface.co/{hugging_user}/{v + c + w}/resolve/main/"
!{aria2c} {huggingface_repo}hubert_base.pt -d hubert -o hubert_base.pt
!{aria2c} {huggingface_repo}rmvpe.pt -d rmvpe -o rmvpe.pt
!{aria2c} {huggingface_repo}uvr5_weights/onnx_dereverb_By_FoxJoy/vocals.onnx -d uvr5_weights/onnx_dereverb_By_FoxJoy -o vocals.onnx
for model in ['D', 'G', 'f0D', 'f0G']:
  !{aria2c} {huggingface_repo}pretrained_v2/{model}48k.pth -d pretrained_v2 -o {model}48k.pth
for model in [
        "HP2-%E4%BA%BA%E5%A3%B0vocals%2B%E9%9D%9E%E4%BA%BA%E5%A3%B0instrumentals",
        "HP2_all_vocals",
        "HP3_all_vocals",
        "HP5-%E4%B8%BB%E6%97%8B%E5%BE%8B%E4%BA%BA%E5%A3%B0vocals%2B%E5%85%B6%E4%BB%96instrumentals",
        "HP5_only_main_vocal",
        "VR-DeEchoAggressive",
        "VR-DeEchoDeReverb",
        "VR-DeEchoNormal",
]:
  !{aria2c} {huggingface_repo}uvr5_weights/{model}.pth -d uvr5_weights -o {model}.pth
%cd ..

!rm -r /content/sample_data

from IPython.display import clear_output
clear_output(wait=True)
print("\n---------------------------\n| Dependencias instaladas |\n---------------------------\n")

In [ ]:
# @title Conectar con Google Drive (Opcional)

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# @title Descargar carpeta de google drive por URL (Opcional)

URL = "https://drive.google.com/drive/folders/1mx2ZjK3hrKHmj8RbAVkaehBVSx7Hie0M" # @param {type:"string"}

!mkdir /content/dataset
%cd /content/dataset
!gdown --folder "{URL}"
%cd /content/rvc

In [ ]:
# @title Configurar modelo
from torch import cuda

NOMBRE_DEL_MODELO = "xadii"  # @param {type:"string"}
NUMERO_DE_HILOS = 2  # @param {type:"slider", min:1, max:16, step:1}

# aliases de variables de español a ingles
MODELNAME = NOMBRE_DEL_MODELO
THREADCOUNT = NUMERO_DE_HILOS

DATASET_DIR = f"logs/{MODELNAME}"
![ -d {DATASET_DIR} ] || mkdir -p {DATASET_DIR}
!echo BATCHSIZE recomendado: {int((cuda.get_device_properties(0).total_memory / 1024 / 1024 / 1024 + 0.4) // 2)}

In [ ]:
# @title Procesar clips de audio

DIRECTORIO_CLIPS_AUDIO = "/content/drive/MyDrive/xado audios/wav pero igual esta MAL" # @param {type: "string"}
PROCESAR_EN_PARALELO = False  # @param {type: "boolean"}
PREPROCESS_PER = "3.0" # @param [ "3.0", "3.7" ]
SAMPLE_RATE = 48000

# aliases de variables de español a ingles
NOPARALLEL = not PROCESAR_EN_PARALELO
PREPROCESS_PER = float(PREPROCESS_PER)

!python infer/modules/train/preprocess.py "{DIRECTORIO_CLIPS_AUDIO}" {SAMPLE_RATE} {THREADCOUNT} {DATASET_DIR} {NOPARALLEL} {PREPROCESS_PER}

In [ ]:
# @title Extraer caracteristicas del modelo

ALGORITMO = "rmvpe_gpu"  # @param [ "pm", "dio", "harvest", "rmvpe", "rmvpe_gpu"]
IS_HALF = True # @param {"type": "boolean"}

if ALGORITMO != "rmvpe_gpu":
  !python infer/modules/train/extract/extract_f0_print.py {DATASET_DIR} {THREADCOUNT} {ALGORITMO}
else:
  !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 {DATASET_DIR} {IS_HALF}
!python infer/modules/train/extract_feature_print.py "cuda" 1 0 0 {DATASET_DIR} "v2" {IS_HALF}

In [ ]:
# @title Entrenar modelo

NUMERO_DE_PASOS = 1000 # @param {type:"integer"}
# @markdown Cada cuantos pasos se deberia guardar el modelo en disco en caso de fallos:
GUARDAR_PASOS = 100 # @param {type:"integer"}
BATCHSIZE = 7  # @param {type:"slider", min:1, max:32, step:1}
# @markdown Guardar cache del entrenamiento en vram (solo recomendado para entrenamientos con menos de 10 minutos de audio)
CACHE_EN_VRAM = False  # @param {type:"boolean"}
# @markdown Solamente guardar el ultimo ckpt para ahorrar espacio en disco
ULTIMO_CKPT = True  # @param {type:"boolean"}

# aliases de variables de español a ingles
MODELEPOCH = NUMERO_DE_PASOS
EPOCHSAVE = GUARDAR_PASOS

# convertir variables del form para ser leibles por los scripts
MODELSAMPLE = "48k"
CACHEDATA = int(CACHE_EN_VRAM)
ONLYLATEST = int(ULTIMO_CKPT)

!python infer/modules/train/train.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g 0 -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained_v2/f0G{MODELSAMPLE}.pth -pd pretrained_v2/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA} -v "v2"